In [69]:
#import dependencies
import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

In [70]:
path = '../weatherpy/outputs/cities.csv'
cities_for_heatmap_df = pd.read_csv(path)
cities_for_heatmap_df.drop(columns='Unnamed: 0', inplace=True)
cities_for_heatmap_df.sample(10)

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement
179,zyryanka,65.75,150.85,-17.09,100,87,11.34,RU,1608823796
110,neryungri,56.67,124.65,-17.25,100,86,4.05,RU,1608823792
104,oranjestad,12.52,-70.03,86.00,20,66,18.34,AW,1608823706
552,medicine hat,50.05,-110.67,35.60,90,55,13.87,CA,1608823847
479,chake chake,-5.25,39.77,81.27,25,77,11.05,TZ,1608823841
420,esperance,-33.87,121.90,68.00,0,64,16.11,AU,1608823836
238,lolodorf,3.23,10.73,79.68,28,81,3.80,CM,1608823799
152,haian,32.54,120.45,44.01,100,73,1.92,CN,1608823794
53,qandala,11.47,49.87,75.52,0,62,8.19,SO,1608823788
359,aras,42.56,-2.36,51.80,75,53,8.05,ES,1608823830


In [71]:
cities_for_heatmap_df['humidity'].max()

100

In [72]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=6)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [73]:
ideal_weather_df = cities_for_heatmap_df[(cities_for_heatmap_df['max_temp'] < 70) & (cities_for_heatmap_df['max_temp'] > 60) & (cities_for_heatmap_df['wind_speed'] < 10) & (cities_for_heatmap_df['cloudiness'] == 0)]
ideal_weather_df['hotel_name'] = " "
ideal_weather_df

<ipython-input-73-b4e20e3c390e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df['hotel_name'] = " "


,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,


In [75]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
api_key=g_key
target_location = -34.54, 20.04
target_radius = 5000
target_keyword = 'hotel'
params = {'key': api_key,
         'location': target_location,
         'radius': target_radius,
         'keyword': target_keyword
         }
data = requests.get(base_url, params=params).json()
data

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [81]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
api_key=g_key
target_location = '-34.54,20.04'
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'

final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
data = requests.get(final_url).json()
data

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -34.5326546, 'lng': 20.0399175},
    'viewport': {'northeast': {'lat': -34.53134607010728,
      'lng': 20.04134342989272},
     'southwest': {'lat': -34.53404572989272, 'lng': 20.03864377010727}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/cafe-71.png',
   'name': 'Bredasdorp Square',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 2976,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109411988681379351778">Coenraad Grunschloss</a>'],
     'photo_reference': 'ATtYBwJ6-DsGj5Wahh_rLwpKSLZOjBIjO3rlVyNBIyZO970sZhUjZ7TSinIs5_jC6FDpuTtG1Qr40J241txndjAuGiG5XRSInU0KAbboytisl-saz-HBwxj9hEOn3SQeq02f3uSUbObyUPqmR8HvlXH9o8zZ5iqhBIch0vOYtpOMaUi5CEoV',
     'width': 3968}],
   'place_id': 'ChIJHf-W89SV0R0R-3OF_h-iDCM',
   'plus_code': {'compound_code': 'F28Q+WX Bredasdorp, South Africa',
    'global_code': '4GQ2F28Q+W

In [ ]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
location = ideal_weather_df[['lat', 'long']]
radius = 5000
keyword = 'hotel'